In [3163]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [3164]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[100,  2])
    
    if v == 1 and csvFile.iloc[100,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[100,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])



[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_15dap_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_his__L_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_ile__L_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_leu__L_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_

In [3165]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= 101

    while i3>100 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<100:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [3166]:


i= 0
indecr_and_stable= list()
diff_indecr= list()
flat= list()
flat2= list()
flat3= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable.append(r_id)

        elif count> 0.95*len(d):
            flat.append(r_id)
        
        else:
            diff_indecr.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable.append(r_id)

        elif count> 1 and len(d) < 3:
            flat.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3.append(r_id)
        
        else:
            diff_indecr.append(r_id)
    
    i= i+1

flat
diff_indecr
indecr_and_stable



['FRD3', 'O2tex', 'SPODM']

In [3167]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
flat2_below_zero= list()
flat3_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero

['EX_csn_e',
 'EX_glc__D_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_h2_e',
 'EX_cynt_e',
 'EX_acgam1p_e',
 'EX_agm_e',
 'EX_alaala_e',
 'EX_man_e',
 'EX_manglyc_e',
 'EX_arg__L_e',
 'EX_o2_e',
 'EX_orn_e',
 'EX_ppa_e',
 'EX_ppal_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_uacgam_e',
 'EX_udpg_e',
 'EX_udpgal_e',
 'EX_fru_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam_e',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'ECA4COLIPAabctex',
 'DSBAO2',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GALabcpp',
 'GLYBabcpp',
 'HYD2pp',
 'HYD3pp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R1bpp',
 'NO3R1pp',
 'NO3R2bpp',
 'NTRIR2x',
 'NTRIR3pp',
 'NTRIR4pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'XYLabcpp',
 'PPPGO',
 'DKMPPD']

In [3168]:
flat3_below_zero

flat_below_zero.append("EX_minohp_e")
flat_below_zero.append("CU1Opp")
len(flat_below_zero)

66

In [3169]:


i2= 0
r_id = indecr_and_stable[0]
mon_incr_and_stable_only_above_zero= list()
mon_decr_and_stable_only_above_zero= list()

for r_id in indecr_and_stable:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_only_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_only_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable)
set2= set(mon_decr_and_stable_only_above_zero)
set3= set(mon_incr_and_stable_only_above_zero)
inANDdecr_and_stable= list(set1-set2-set3)
inANDdecr_and_stable


['O2tex']

In [3170]:
i2= 0
r_id = indecr_and_stable_below_zero[0]
mon_incr_and_stable_only_below_zero= list()
mon_decr_and_stable_only_below_zero= list()

for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_only_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_only_below_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_only_below_zero)
set3= set(mon_incr_and_stable_only_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['EX_glc__D_e',
 'EX_udpg_e',
 'EX_o2_e',
 'EX_uacgam_e',
 'PPPGO',
 'NO3R1bpp',
 'NTRIR3pp',
 'NTRIR4pp',
 'EX_gam_e',
 'NTRIR2x',
 'EX_fru_e',
 'EX_agm_e',
 'EX_udpgal_e',
 'EX_ppal_e',
 'EX_ppa_e',
 'EX_manglyc_e',
 'EX_h2_e',
 'EX_cynt_e',
 'EX_man_e']

In [3171]:
csvFile = pd.read_csv('biomass_per_flux_for_reaction_EX_h2_e.csv')
l= list()
i=0

while i < 201:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile2= pd.DataFrame (l, columns = ['Biomass'])
csvFile2.Biomass = csvFile2.Biomass.round(2)
l= list(csvFile2.Biomass)

my_series = csvFile2['Biomass'].squeeze()

n2=0

if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and l[n2+4] == max(l):
    print("no")

else:
    print("yes")

    
n2= len(l)-1

if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and l[n2-4] < max(l) or l[n2-4] == max(l):
    print("no")

else:
    print("yes")

no
no


In [3172]:
w= list()
w2= list()
i2=0

while i2 < len(indecr_and_stable_below_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w2.append(r_id)

    
    i2= i2+1

w

['NTRIR3pp', 'NTRIR4pp', 'PPPGO']

In [3173]:
w2

['EX_o2_e']

In [3174]:
inANDdecr_and_stable_below_zero.remove("NTRIR4pp")
inANDdecr_and_stable_below_zero.remove("PPPGO")
inANDdecr_and_stable_below_zero.remove("NTRIR3pp")
inANDdecr_and_stable_below_zero.remove("EX_o2_e")
inANDdecr_and_stable_below_zero


['EX_glc__D_e',
 'EX_udpg_e',
 'EX_uacgam_e',
 'NO3R1bpp',
 'EX_gam_e',
 'NTRIR2x',
 'EX_fru_e',
 'EX_agm_e',
 'EX_udpgal_e',
 'EX_ppal_e',
 'EX_ppa_e',
 'EX_manglyc_e',
 'EX_h2_e',
 'EX_cynt_e',
 'EX_man_e']

In [3175]:
mon_decr_and_stable_only_below_zero.append("NTRIR4pp")
mon_decr_and_stable_only_below_zero.append("PPPGO")
mon_decr_and_stable_only_below_zero.append("NTRIR3pp")
mon_incr_and_stable_only_below_zero.append("EX_o2_e")
mon_decr_and_stable_only_below_zero



['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_acgam1p_e',
 'EX_alaala_e',
 'EX_arg__L_e',
 'EX_orn_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GALabcpp',
 'GLYBabcpp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R2bpp',
 'XYLabcpp',
 'DKMPPD',
 'NTRIR4pp',
 'PPPGO',
 'NTRIR3pp']

In [3176]:
w3= list()
w4= list()
i2=0

while i2 < len(indecr_and_stable):
    
    i=0
    r_id = indecr_and_stable[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w3.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w4.append(r_id)

    
    i2= i2+1

w3

['O2tex']

In [3177]:
inANDdecr_and_stable.remove("O2tex")
mon_decr_and_stable_only_above_zero.append("O2tex")
mon_decr_and_stable_only_above_zero



['FRD3', 'O2tex']

In [3178]:
i= 0
indecr_and_stable_both_positive_and_negative= list()
diff_indecr_both_positive_and_negative= list()
flat_both_positive_and_negative= list()
flat2_both_positive_and_negative= list()
flat3_both_positive_and_negative= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l3 and r_id in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_both_positive_and_negative.append(r_id)

        elif count> 0.95*len(d):
            flat_both_positive_and_negative.append(r_id)
        
        else:
            diff_indecr_both_positive_and_negative.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_both_positive_and_negative.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_both_positive_and_negative.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_both_positive_and_negative.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_both_positive_and_negative.append(r_id)
        
        else:
            diff_indecr_both_positive_and_negative.append(r_id)
    
    i= i+1

flat_both_positive_and_negative
diff_indecr_both_positive_and_negative
indecr_and_stable_both_positive_and_negative


['EX_glcn_e',
 'EX_glyclt_e',
 'EX_h_e',
 'EX_h2o_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'EX_ac_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'EX_for_e',
 'ACKr',
 'ALAALAD',
 'ADA',
 'ADD',
 'ALATA_D2',
 'ALATA_L2',
 'ALAabcpp',
 'ALDD2x',
 'ADNK1',
 'ADNUC',
 'ALDD2y',
 'ADPT',
 'AMPN',
 'ASPabcpp',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'ASPt2pp_copy2',
 'ATPM',
 'CITL',
 'CBPS',
 'CD2abcpp',
 'CMtpp',
 'COBALT2abcpp',
 'CRNDabcpp',
 'CRNDt2rpp',
 'CRNabcpp',
 'DADA',
 'CTBTabcpp',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'DHAPT',
 'ARBabcpp',
 'ARBt3ipp',
 'DHORD5',
 'DOXRBCNtpp',
 'ARGDCpp',
 'ARGabcpp',
 'F6PP',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'FDH4pp',
 'ASNN',
 'FDH5pp',
 'FACOAL60t2pp',
 'ASNNpp',
 'ASNS1',
 'FEROpp',
 'FE2abcpp',
 'FE2t2pp',
 'FE2tex',
 'ASNabcpp',
 'FRD2',
 'FE3tex',
 'FTHFLi',
 'FUSAtpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'G6PP',
 'GLCabcpp',
 'ASPT',
 'GLUN',
 'GART',
 'GDPMNP',
 'GLUSy',
 'GLUabcpp',
 'GLYCK2',
 'GUAt2pp',
 'GLYC3Pabcpp',
 'H2O

In [3179]:
i2= 0
r_id = indecr_and_stable_both_positive_and_negative[0]
mon_incr_and_stable_both_positive_and_negative= list()
mon_decr_and_stable_both_positive_and_negative= list()

for r_id in indecr_and_stable_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_both_positive_and_negative)
set2= set(mon_decr_and_stable_both_positive_and_negative)
set3= set(mon_incr_and_stable_both_positive_and_negative)
inANDdecr_and_stable_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_and_stable_both_positive_and_negative


['GLYCTO4',
 'PIuabcpp',
 'PYRDC',
 'PTAr',
 'Kabcpp',
 'PYK',
 'EX_h_e',
 'EX_fe3_e',
 'ADNK1',
 'NADH16pp',
 'ZNabcpp',
 'GSNK',
 'MNt2pp',
 'FEROpp',
 'HXCT',
 'ARGDCpp',
 'PPA',
 'SULabcpp',
 'ACKr',
 'PRPPS',
 'NTD9',
 'H2Otex',
 'INSK',
 'ASNS1',
 'HEX7',
 'Htex',
 'NADH17pp',
 'ALATA_L2',
 'GLYCTO3',
 'GLYCK2',
 'FRD2',
 'EX_for_e',
 'GART',
 'EX_h2o_e',
 'CITL',
 'ASNN',
 'DHORD5',
 'NTD8',
 'EX_ac_e',
 'PFK_3',
 'GTPDPDP',
 'NADH18pp',
 'FDH4pp',
 'NI2uabcpp',
 'GLUSy',
 'H2SO',
 'FDH5pp',
 'PDH',
 'GLYCTO2',
 'CBPS',
 'URIK2']

In [3180]:
w5= list()
w6= list()
i2=0

while i2 < len(inANDdecr_and_stable_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    i2= i2+1
    
w5

['PIuabcpp',
 'PYRDC',
 'Kabcpp',
 'EX_h_e',
 'ADNK1',
 'ZNabcpp',
 'GSNK',
 'MNt2pp',
 'FEROpp',
 'HXCT',
 'ARGDCpp',
 'PPA',
 'SULabcpp',
 'ACKr',
 'PRPPS',
 'NTD9',
 'INSK',
 'ASNS1',
 'HEX7',
 'ALATA_L2',
 'GLYCK2',
 'EX_for_e',
 'GART',
 'NTD8',
 'PFK_3',
 'GTPDPDP',
 'FDH4pp',
 'NI2uabcpp',
 'GLUSy',
 'H2SO',
 'PDH',
 'GLYCTO2',
 'CBPS',
 'URIK2']

In [3181]:
w6

['PTAr', 'EX_fe3_e', 'FEROpp', 'Htex', 'DHORD5']

In [3182]:
len(inANDdecr_and_stable_both_positive_and_negative)

51

In [3183]:
inANDdecr_and_stable_both_positive_and_negative.remove("PTAr")
inANDdecr_and_stable_both_positive_and_negative.remove("Htex")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_fe3_e")
inANDdecr_and_stable_both_positive_and_negative.remove("DHORD5")
inANDdecr_and_stable_both_positive_and_negative.remove("ASNS1")
inANDdecr_and_stable_both_positive_and_negative.remove("FDH4pp")
inANDdecr_and_stable_both_positive_and_negative.remove("GTPDPDP")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_for_e")
inANDdecr_and_stable_both_positive_and_negative.remove("FEROpp")
inANDdecr_and_stable_both_positive_and_negative.remove("ACKr")
inANDdecr_and_stable_both_positive_and_negative.remove("CBPS")
inANDdecr_and_stable_both_positive_and_negative.remove("PFK_3")
inANDdecr_and_stable_both_positive_and_negative.remove("PIuabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("GART")
inANDdecr_and_stable_both_positive_and_negative.remove("NTD8")
inANDdecr_and_stable_both_positive_and_negative.remove("NI2uabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("GLYCTO2")
inANDdecr_and_stable_both_positive_and_negative.remove("SULabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("PPA")
inANDdecr_and_stable_both_positive_and_negative.remove("PDH")
inANDdecr_and_stable_both_positive_and_negative.remove("GLYCK2")
inANDdecr_and_stable_both_positive_and_negative.remove("GLUSy")
inANDdecr_and_stable_both_positive_and_negative.remove("PYRDC")
inANDdecr_and_stable_both_positive_and_negative.remove("ALATA_L2")
inANDdecr_and_stable_both_positive_and_negative.remove("Kabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("HXCT")
inANDdecr_and_stable_both_positive_and_negative.remove("GSNK")
inANDdecr_and_stable_both_positive_and_negative.remove("INSK")
inANDdecr_and_stable_both_positive_and_negative.remove("ADNK1")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_h_e")
inANDdecr_and_stable_both_positive_and_negative.remove("PRPPS")
inANDdecr_and_stable_both_positive_and_negative.remove("NTD9")
inANDdecr_and_stable_both_positive_and_negative.remove("URIK2")
inANDdecr_and_stable_both_positive_and_negative.remove("ZNabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("ARGDCpp")
inANDdecr_and_stable_both_positive_and_negative.remove("HEX7")
inANDdecr_and_stable_both_positive_and_negative.remove("H2SO")
inANDdecr_and_stable_both_positive_and_negative.remove("MNt2pp")



In [3184]:
flat_both_positive_and_negative.append("FEROpp")
increasing_below_zero_and_flat_above_zero= ["EX_fe3_e", "PTAr", "Htex"]
increasing_and_stable_below_zero_and_decreasing_above_zero= ["DHORD5"]
flat_below_zero_and_decreasing_above_zero= ["ASNS1", "FDH4pp", "GTPDPDP", "ACKr", "PFK_3", "GART", "NTD8", "GLYCTO2", "PDH",  "GLYCK2", "GLUSy", "PYRDC", "ALATA_L2", "HXCT", "GSNK", "INSK", "ADNK1", "EX_h_e", "PRPPS", "NTD9", "URIK2"]
decreasing_and_stable_below_zero_and_decreasing_above_zero= ["EX_for_e", "CBPS", "PIuabcpp", "NI2uabcpp", "SULabcpp", "PPA", "Kabcpp", "ZNabcpp", "ARGDCpp", "HEX7", "H2SO", "MNt2pp"]



In [3185]:
inANDdecr_and_stable_both_positive_and_negative

['GLYCTO4',
 'PYK',
 'NADH16pp',
 'H2Otex',
 'NADH17pp',
 'GLYCTO3',
 'FRD2',
 'EX_h2o_e',
 'CITL',
 'ASNN',
 'EX_ac_e',
 'NADH18pp',
 'FDH5pp']

In [3186]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()


while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        A.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            decreasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        elif len(csvFile2['Biomass'].unique()) < 3:
            flat_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    i=i+1

decreasing_and_stable_below_zero_and_decreasing_above_zero

['EX_for_e',
 'CBPS',
 'PIuabcpp',
 'NI2uabcpp',
 'SULabcpp',
 'PPA',
 'Kabcpp',
 'ZNabcpp',
 'ARGDCpp',
 'HEX7',
 'H2SO',
 'MNt2pp',
 'EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'ALAALAD',
 'ALAabcpp',
 'ALDD2x',
 'ALDD2y',
 'ASPabcpp',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'ATPM',
 'CD2abcpp',
 'CMtpp',
 'COBALT2abcpp',
 'CRNDabcpp',
 'CRNabcpp',
 'CTBTabcpp',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'ARBabcpp',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'ARGabcpp',
 'FE2abcpp',
 'FE2t2pp',
 'ASNabcpp',
 'FUSAtpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'GLCabcpp',
 'GLUabcpp',
 'GUAt2pp',
 'GLYC3Pabcpp',
 'ILEabcpp',
 'HISabcpp',
 'LCTSt3ipp',
 'LDH_D2',
 'LEUabcpp',
 'LYSabcpp',
 'MALt2_3pp',
 'MINCYCtpp',
 'MDH2',
 'MDH3',
 'MG2tpp',
 'MG2uabcpp',
 'NADH10',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2abcpp',
 'GTHPi',
 'NTP1',
 'NTP10',
 'NTP3',
 'NTP5',
 'NTPP1',
 'NTPP2',
 'NTPP3',
 'NTPP4',
 'NTPP5',
 'NTPP6',
 'NTPP7',
 'NTPP8',
 

In [3187]:
W= list(decreasing_and_stable_below_zero_and_decreasing_above_zero)+list(flat_below_zero_and_decreasing_above_zero)

In [3188]:
union = pd.Series(np.union1d(A, W))
intersect = pd.Series(np.intersect1d(A, W))
BBB= union[~union.isin(intersect)]
BBB


0           ACKr
3          ADNK1
8       ALATA_L2
15       ARGDCpp
18         ASNS1
26          CBPS
42      EX_for_e
45        EX_h_e
54        FDH4pp
64          GART
68         GLUSy
71        GLYCK2
73       GLYCTO2
75          GSNK
77       GTPDPDP
80          H2SO
82          HEX7
85          HXCT
90          INSK
91        Kabcpp
114       MNt2pp
125    NI2uabcpp
128         NTD8
129         NTD9
146          PDH
148        PFK_3
149     PIuabcpp
151          PPA
156        PRPPS
158        PYRDC
165     SULabcpp
173        URIK2
179      ZNabcpp
dtype: object

In [3189]:
z=0
f= list()
g= list()
h= list()
i= list()
j= list()
flat_below_zero_and_increasing_above_zero= list()

while z < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        f.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            increasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        else:
            flat_below_zero_and_increasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        g.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        h.append(mon_incr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        i.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        j.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    z=z+1

increasing_and_stable_below_zero_and_decreasing_above_zero

['DHORD5']

In [3190]:
h

['CRNDt2rpp', 'FE2tex', 'DURAD']

In [3191]:
increasing_below_zero_and_flat_above_zero.append("CRNDt2rpp")
increasing_below_zero_and_flat_above_zero.append("FE2tex")
increasing_below_zero_and_flat_above_zero.append("DURAD")

increasing_below_zero_and_flat_above_zero



['EX_fe3_e', 'PTAr', 'Htex', 'CRNDt2rpp', 'FE2tex', 'DURAD']

In [3192]:
z=0
k= list()
L= list()
m= list()
o= list()
p= list()
increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero= list()
flat_below_zero_and_increasingANDdecreasing_above_zero= list()

while z < len(inANDdecr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        k.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        else:
            flat_below_zero_and_increasingANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        L.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        m.append(inANDdecr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        o.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        p.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    z=z+1

increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero

['GLYCTO4', 'H2Otex', 'GLYCTO3', 'CITL', 'ASNN', 'EX_ac_e', 'FDH5pp']

In [3193]:
p

increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= ["NADH16pp"]
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero




['NADH16pp']

In [3194]:
increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero= ["H2Otex", "EX_ac_e"]
increasing_and_stable_below_zero_and_decreasing_above_zero.append("GLYCTO4")
increasing_and_stable_below_zero_and_decreasing_above_zero.append("GLYCTO3")
increasing_and_stable_below_zero_and_decreasing_above_zero.append("FDH5pp")
flat_below_zero_and_decreasing_above_zero.append("ASNN")
increasing_below_zero_and_decreasing_above_zero= ["CITL"]




In [3195]:
L

increasing_below_zero_and_decreasing_above_zero.append("PYK")



In [3196]:
m

increasing_below_zero_and_decreasing_and_stable_above_zero= list()
increasing_below_zero_and_decreasing_and_stable_above_zero= ["NADH18pp", "FRD2", "NADH17pp"]
increasing_below_zero_and_increasing_and_decreasing_and_stable_above_zero= ["EX_h2o_e"]



In [3197]:
i2= 0
r_id = diff_indecr[0]
incr_above_zero= list()
decr_above_zero= list()
one_to_three_data_point= list()

for r_id in diff_indecr:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point.append(r_id)

    
    i2= i2 + 1

decr_above_zero


['2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD120',
 '3HAD140',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 'ABTA',
 'ABUTD',
 '3OAR120',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS120',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'ACPPAT180',
 'ACPPAT181',
 'ADNCYC',
 'ADPRDP',
 'AGDC',
 'AGMT',
 'AGPAT120',
 'AGPAT140',
 'AOBUTDs',
 'AGPAT141',
 'AMPTASECG',
 'AGPAT180',
 'AP4AH',
 'AP4AS',
 'APG3PAT120',
 'APG3PAT140',
 'APG3PAT141',
 'AST',
 'ATPHs',
 'APG3PAT180',
 'CGLYabcpp',
 'APG3PAT181

In [3198]:
incr_above_zero

[]

In [3199]:
set1= set(diff_indecr)
set2= set(decr_above_zero)
set3= set(incr_above_zero)
set4= set(one_to_three_data_point)
inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_above_zero




['PSD160',
 'O2tpp',
 'GMPS2',
 'ACGK',
 'AGPAT160',
 'ACONTb',
 'ALAR',
 '3OAR140',
 'GF6PTA',
 'PSD161',
 'CU2tpp',
 'ACGS',
 'ADSS',
 'P5CR',
 'HCO3E',
 'NH4tpp',
 'NDPK2',
 'CYSS',
 'DTMPK',
 'ASPK',
 'AGPAT161',
 'GK1',
 'UAGDP',
 'G1PACT',
 'CS',
 'NH4tex',
 'ALAALAr',
 'DASYN160',
 'NADK',
 'DASYN161',
 'ACPPAT160',
 'UMPK',
 'SERAT',
 'CTPS2',
 'ACODA',
 'THRD_L',
 'ADSL1r',
 'MCOATA',
 'THRS',
 'G5SADs',
 'TMPK',
 'PMPK',
 'NNATr',
 '3OAS140',
 'ARGSS',
 'GLNS',
 'ICDHyr',
 'NDPK4',
 'ACONTa',
 'OCBT',
 'HSK',
 'ARGSL',
 'ACCOAC',
 'NADS1',
 'PSSA161',
 'ACPPAT161',
 'APG3PAT161',
 'APG3PAT160',
 'PE161abcpp',
 'PE160abcpp',
 'COBALT2tpp',
 'PSSA160']

In [3200]:
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w7

['PSD160',
 'O2tpp',
 'GMPS2',
 'ACGK',
 'AGPAT160',
 'ACONTb',
 'ALAR',
 '3OAR140',
 'GF6PTA',
 'PSD161',
 'CU2tpp',
 'ACGS',
 'ADSS',
 'P5CR',
 'HCO3E',
 'NH4tpp',
 'NDPK2',
 'CYSS',
 'DTMPK',
 'ASPK',
 'AGPAT161',
 'GK1',
 'UAGDP',
 'G1PACT',
 'CS',
 'NH4tex',
 'ALAALAr',
 'DASYN160',
 'NADK',
 'DASYN161',
 'ACPPAT160',
 'UMPK',
 'SERAT',
 'CTPS2',
 'ACODA',
 'THRD_L',
 'ADSL1r',
 'MCOATA',
 'THRS',
 'G5SADs',
 'TMPK',
 'PMPK',
 'NNATr',
 '3OAS140',
 'ARGSS',
 'GLNS',
 'ICDHyr',
 'NDPK4',
 'ACONTa',
 'OCBT',
 'HSK',
 'ARGSL',
 'ACCOAC',
 'NADS1',
 'PSSA161',
 'ACPPAT161',
 'APG3PAT161',
 'APG3PAT160',
 'PE161abcpp',
 'PE160abcpp',
 'COBALT2tpp',
 'PSSA160']

In [3201]:
set1= set(inANDdecr_above_zero)
set2= set(w7)
set3= set(w8)
truly_inANDdecr_above_zero= list(set1-set2-set3)
truly_inANDdecr_above_zero



[]

In [3202]:
ww= w7 + w8
ww

ww = list(set(ww))

decr_above_zero= decr_above_zero + ww
decr_above_zero

['2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD120',
 '3HAD140',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 'ABTA',
 'ABUTD',
 '3OAR120',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS120',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'ACPPAT180',
 'ACPPAT181',
 'ADNCYC',
 'ADPRDP',
 'AGDC',
 'AGMT',
 'AGPAT120',
 'AGPAT140',
 'AOBUTDs',
 'AGPAT141',
 'AMPTASECG',
 'AGPAT180',
 'AP4AH',
 'AP4AS',
 'APG3PAT120',
 'APG3PAT140',
 'APG3PAT141',
 'AST',
 'ATPHs',
 'APG3PAT180',
 'CGLYabcpp',
 'APG3PAT181

In [3203]:
i2= 0
r_id = diff_indecr_below_zero[0]
incr_below_zero= list()
decr_below_zero= list()
one_to_three_data_point2= list()

for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_below_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point2.append(r_id)

    
    i2= i2 + 1

decr_below_zero


['EX_cmp_e',
 'EX_glcur1p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_23camp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_2ddglcn_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_cytd_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_lcts_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_mal__D_e',
 'EX_dgmp_e',
 'EX_man6p_e',
 'EX_amp_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_pser__L_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_f6p_e',
 'EX_thrp_e',
 'EX_thymd_e',
 'EX_tre_e',
 'EX_trp__L_e',
 'EX_ump_e',
 'EX_uri_e',
 'EX_g3pc_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e']

In [3204]:
incr_below_zero

['EX_cpgn_e',
 'EX_isetac_e',
 'EX_arbtn_fe3_e',
 'EX_mso3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_orot_e',
 'EX_sulfac_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 '2DGULRGx',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feenter_e',
 'EX_feoxam_e',
 'ACOATA',
 'ADK4',
 'ALPATE160pp',
 'ALPATG160pp',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CTBTt2rpp',
 'CUt3',
 'ARGAGMt7pp',
 'DOGULNR',
 'EAR60x',
 'EAR80x',
 'DSERDHr',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'EAR40x',
 'FESD2s',
 'FORtex',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LPLIPAL2ATE120',
 'LPLIPAL2ATE140',
 'LPLIPAL2ATE141',
 'LPLIPAL2ATE160',
 'LPLIPAL2ATE161',
 'LPLIPAL2ATE180',
 'LPLIPAL2ATE181',
 'LPLIPAL2ATG120',
 'LPLIPAL2ATG140',
 'LPLIPAL2ATG141',
 'LPLIPAL2ATG160',
 'LPLIPAL2ATG161',
 'LPLIPAL2ATG180',
 'LPLIPAL2ATG181',
 'LEUt2rpp',
 'LIPOCT',
 'GPDDA3pp',
 

In [3205]:
set1= set(diff_indecr_below_zero)
set2= set(decr_below_zero)
set3= set(incr_below_zero)
set4= set(one_to_three_data_point2)
inANDdecr_below_zero= list(set1-set2-set3-set4)
inANDdecr_below_zero


['EX_gal_e',
 'EX_butso3_e',
 'EX_gthrd_e',
 'EX_xylu__L_e',
 'EX_ddca_e',
 'OMMBLHX3',
 'EX_xyl__D_e',
 'EX_octa_e',
 'OMPHHX3',
 'EX_galct__D_e',
 'EX_rib__D_e',
 'EX_udpglcur_e',
 'EX_LalaDglu_e',
 'EX_hdca_e',
 'EX_LalaDgluMdapDala_e',
 'EX_cgly_e',
 'EX_ocdcea_e',
 'PGAMT',
 'G3PD2',
 'EX_23dappa_e',
 'EX_23ccmp_e',
 'RPI',
 'OPHHX3',
 'EX_acmum_e',
 'EX_acnam_e',
 'EX_malt_e',
 'EX_progly_e',
 'EX_3hcinnm_e',
 'EX_ocdca_e',
 'EX_udpacgal_e',
 'EX_anhgm_e',
 'AGPR',
 'EX_dca_e',
 'EX_chtbs_e',
 'EX_glcr_e',
 'EX_galt_e',
 'EX_mnl_e',
 'EX_tartr__D_e',
 'EX_LalaLglu_e',
 'EX_tartr__L_e',
 'EX_lyx__L_e',
 'EX_nh4_e',
 'EX_galur_e',
 'EX_all__D_e',
 'EX_LalaDgluMdap_e',
 'EX_arab__L_e',
 'EX_sbt__D_e',
 'EX_ascb__L_e',
 'EX_sucr_e',
 'EX_acgam_e',
 'EX_acmana_e',
 'ASAD',
 'ACOTA',
 'EX_ttdcea_e',
 'EX_melib_e',
 'ASPTA',
 'EX_ethso3_e',
 'EX_ptrc_e',
 'EX_pppn_e',
 'EX_3hpppn_e',
 'EX_fum_e',
 'EX_26dap__M_e',
 'EX_fe3dcit_e',
 'EX_but_e',
 'EX_ttdca_e',
 'HSDy',
 'EX_hxa_e',
 'EX_h

In [3206]:
w9= list()
w10= list()
i2=0

while i2 < len(inANDdecr_below_zero):
    
    i=0
    r_id = inANDdecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w9.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w10.append(r_id)
    
    i2= i2+1
    
w9

['PGAMT']

In [3207]:
w10

['OMMBLHX3',
 'OMPHHX3',
 'PGAMT',
 'G3PD2',
 'RPI',
 'OPHHX3',
 'AGPR',
 'EX_nh4_e',
 'ASAD',
 'ACOTA',
 'ASPTA',
 'EX_26dap__M_e',
 'HSDy']

In [3208]:
set1= set(inANDdecr_below_zero)
set2= set(w9)
set3= set(w10)
truly_inANDdecr_below_zero= list(set1-set2-set3)
truly_inANDdecr_below_zero


['EX_gal_e',
 'EX_butso3_e',
 'EX_xyl__D_e',
 'EX_galct__D_e',
 'EX_udpglcur_e',
 'EX_LalaDglu_e',
 'EX_cgly_e',
 'EX_ocdcea_e',
 'EX_acmum_e',
 'EX_anhgm_e',
 'EX_dca_e',
 'EX_chtbs_e',
 'EX_galt_e',
 'EX_mnl_e',
 'EX_tartr__D_e',
 'EX_galur_e',
 'EX_all__D_e',
 'EX_LalaDgluMdap_e',
 'EX_sbt__D_e',
 'EX_ascb__L_e',
 'EX_sucr_e',
 'EX_melib_e',
 'EX_ethso3_e',
 'EX_fe3dcit_e',
 'EX_hdcea_e',
 'EX_gthrd_e',
 'EX_xylu__L_e',
 'EX_ddca_e',
 'EX_octa_e',
 'EX_rib__D_e',
 'EX_hdca_e',
 'EX_LalaDgluMdapDala_e',
 'EX_23dappa_e',
 'EX_23ccmp_e',
 'EX_acnam_e',
 'EX_malt_e',
 'EX_progly_e',
 'EX_3hcinnm_e',
 'EX_ocdca_e',
 'EX_udpacgal_e',
 'EX_glcr_e',
 'EX_LalaLglu_e',
 'EX_tartr__L_e',
 'EX_lyx__L_e',
 'EX_arab__L_e',
 'EX_acgam_e',
 'EX_acmana_e',
 'EX_ttdcea_e',
 'EX_3hpppn_e',
 'EX_ptrc_e',
 'EX_pppn_e',
 'EX_fum_e',
 'EX_but_e',
 'EX_ttdca_e',
 'EX_hxa_e']

In [3209]:
ww= w9 + w10
ww

ww = list(set(ww))

incr_below_zero= incr_below_zero + ww
incr_below_zero


['EX_cpgn_e',
 'EX_isetac_e',
 'EX_arbtn_fe3_e',
 'EX_mso3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_orot_e',
 'EX_sulfac_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 '2DGULRGx',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feenter_e',
 'EX_feoxam_e',
 'ACOATA',
 'ADK4',
 'ALPATE160pp',
 'ALPATG160pp',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CTBTt2rpp',
 'CUt3',
 'ARGAGMt7pp',
 'DOGULNR',
 'EAR60x',
 'EAR80x',
 'DSERDHr',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'EAR40x',
 'FESD2s',
 'FORtex',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LPLIPAL2ATE120',
 'LPLIPAL2ATE140',
 'LPLIPAL2ATE141',
 'LPLIPAL2ATE160',
 'LPLIPAL2ATE161',
 'LPLIPAL2ATE180',
 'LPLIPAL2ATE181',
 'LPLIPAL2ATG120',
 'LPLIPAL2ATG140',
 'LPLIPAL2ATG141',
 'LPLIPAL2ATG160',
 'LPLIPAL2ATG161',
 'LPLIPAL2ATG180',
 'LPLIPAL2ATG181',
 'LEUt2rpp',
 'LIPOCT',
 'GPDDA3pp',
 

In [3210]:
i2= 0
r_id = diff_indecr_both_positive_and_negative[0]
incr_both_positive_and_negative= list()
decr_both_positive_and_negative= list()
one_to_three_data_point3= list()

for r_id in diff_indecr_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_both_positive_and_negative.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point3.append(r_id)

    
    i2= i2 + 1

decr_both_positive_and_negative

['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e']

In [3211]:
incr_both_positive_and_negative

['GLCptspp']

In [3212]:
one_to_three_data_point3

['FACOAE80',
 'GNK',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp']

In [3213]:
decreasing_below_zero_and_decreasing_above_zero= ["EX_glu__L_e", "EX_gly_e", "EX_glyald_e", "EX_glyc3p_e", "EX_5dglcn_e", "EX_acald_e", "EX_lac__D_e", "EX_lac__L_e", "EX_akg_e", "EX_ala__L_e", "EX_mal__L_e", "EX_asp__L_e", "EX_dha_e", "EX_pyr_e", "EX_ser__L_e", "EX_etha_e", "EX_thr__L_e"]
increasing_below_zero_and_increasing_above_zero= ["GLCptspp"]


In [3214]:
set1= set(diff_indecr_both_positive_and_negative)
set2= set(decr_both_positive_and_negative)
set3= set(incr_both_positive_and_negative)
set4= set(one_to_three_data_point3)
inANDdecr_both_positive_and_negative= list(set1-set2-set3-set4)
inANDdecr_both_positive_and_negative


['ARBTNR1',
 'EX_val__L_e',
 '4PEPTabcpp',
 'APH141',
 'AACPS2',
 'MALt2_2pp',
 'XYLI2',
 'PGL',
 'PGPP161',
 'XPPT',
 'GLYCtpp',
 'FUM',
 'DURIK1',
 '2AGPEAT161',
 'FEENTERR1',
 '2AGPGAT140',
 'FTHFD',
 'EAR140x',
 'MGSA',
 'LALDO2x',
 '2AGPEAT181',
 'AM4PCP',
 'EAR141x',
 'GDPDPK',
 '2AGPEAT120',
 'RNDR4b',
 'RNTR3c2',
 'CLPNH160pp',
 'FBA',
 'AGM3Pt2pp',
 'ENO',
 'APH180',
 'CYStpp',
 'CLPNH140pp',
 'PGPP140',
 'PA160abcpp',
 'EX_pro__L_e',
 'CO2tpp',
 'FEENTERR2',
 'GLCt2pp',
 'AGM3PA',
 'GHMT2r',
 'PUNP3',
 'MLTGY2pp',
 'ZN2tpp',
 'AGM4PCPpp',
 'CBMKr',
 'GLUDC',
 'PA141abcpp',
 'RNTR4c',
 'PGK',
 '2AGPEAT160',
 'PFL',
 'FBP',
 'NDPK8',
 'FE3HOXR1',
 'NTD2',
 'AGM3PApp',
 'CYTDK2',
 'SUCCt2_2pp',
 'EX_succ_e',
 'EAR180x',
 'MTHFC',
 'ICL',
 'RNTR1c',
 '2AGPEAT180',
 'PA161abcpp',
 'ENTERES2',
 'IDOND',
 'CDAPPA181',
 '2AGPEAT141',
 'PUNP2',
 'EX_ade_e',
 'TKT1',
 'APH161',
 'UM3PL',
 'NTD11',
 'NACODA',
 'CYSDS',
 'TRE6PH',
 'GMPR',
 'FACOAE181',
 'HXAND',
 'PA180abcpp',
 'EX_glyc

In [3215]:
w11= list()
w12= list()
i2=0

while i2 < len(inANDdecr_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w11.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w12.append(r_id)
    
    i2= i2+1
    
w11


['EX_val__L_e',
 '4PEPTabcpp',
 'AACPS2',
 'PGL',
 'XPPT',
 'GLYCtpp',
 'FUM',
 'AM4PCP',
 'AGM3Pt2pp',
 'FEENTERR2',
 'AGM3PA',
 'MLTGY2pp',
 'AGM4PCPpp',
 'PGK',
 'AGM3PApp',
 'EX_succ_e',
 'ICL',
 'TKT1',
 'CYSDS',
 'GMPR',
 'AGM4PCP',
 'ARBTNR3',
 'ALAALAabcpp',
 'FEOXAMR3',
 'FECRMR3',
 'AACPS5',
 'NTD1',
 'G3PEabcpp',
 'AACPS6',
 'FECRMR2',
 'CYSabcpp',
 'TMDK1',
 'DURIPP',
 'FA80ACPHi',
 'FE3HOXR3',
 'P5CD',
 'AGM3PH',
 'PPS',
 'AM4PA',
 'AACPS4',
 'EX_ura_e',
 'AACPS3',
 '4PCPpp',
 'DALAt2pp',
 'FEENTERR3',
 'FACOAE141',
 'ETHAt2pp',
 'GPDDA4pp',
 'PROD2',
 'EX_ala__D_e',
 'FA100ACPHi',
 '4PCP',
 'CPGNR2',
 'FEOXAMR2',
 'FE3HOXR2',
 'AACPS7',
 'AACPS1',
 'CPGNR3',
 'SUCDi',
 'ARBTNR2',
 'TKT2']

In [3216]:
union = pd.Series(np.union1d(w11, w12))
set1= set(inANDdecr_both_positive_and_negative)
set2= set(union)
truly_inANDdecr_both_positive_and_negative= list(set1-set2)
truly_inANDdecr_both_positive_and_negative


['FE3abcpp',
 'SSALy',
 'APH141',
 'SUCMALtpp',
 '2AGPGAT181',
 'NMNDA',
 'FACOAE180',
 'RNDR3',
 'NMNAT',
 'GUAD',
 '2AGPGAT160',
 'MALt2_2pp',
 'XYLI2',
 'NI2tpp',
 'DURIK1',
 '2AGPEAT161',
 'AKGDH',
 'MOX',
 '2AGPGAT140',
 'FTHFD',
 'MGSA',
 'NI2t3pp',
 '2AGPEAT181',
 'H2Otpp',
 'RNTR2c',
 'GDPDPK',
 '2AGPEAT120',
 'RNDR4b',
 'RNTR3c2',
 'GDPMNH',
 'CDAPPA141',
 '2AGPGAT120',
 'FBA',
 'ENO',
 'CDAPPA140',
 'CYTK1',
 'APH180',
 'CYStpp',
 'APH140',
 'RNDR1b',
 'GDPTPDP',
 'NMNN',
 'RPE',
 'PA160abcpp',
 'CAT',
 'EX_pro__L_e',
 'ASNS2',
 'CO2tpp',
 'ORNDC',
 'PUNP3',
 'APH120',
 'ZN2tpp',
 'CBMKr',
 'GLUDC',
 'MDH',
 'PA141abcpp',
 'RNTR4c',
 'PFL',
 '2AGPEAT160',
 'FBP',
 'PGI',
 'NDPK8',
 'GLUDy',
 'PUNP1',
 'NTD2',
 'RNTR4c2',
 'CYTDK2',
 'SUCCt2_2pp',
 'LDH_D',
 'MTHFC',
 'GLYCL',
 '2AGPGAT180',
 'EX_4abut_e',
 'RNTR1c',
 'GND',
 '2AGPEAT180',
 'GLYAT',
 'PPK2',
 'RNTR1c2',
 'PA161abcpp',
 'CO2tex',
 'RNTR3c',
 'IMPD',
 'CDAPPA181',
 '2AGPEAT141',
 'PUNP2',
 'EX_glyc_e',
 'FACOAE1

In [3217]:
union = pd.Series(np.union1d(w11, w12))
intersect = pd.Series(np.intersect1d(w11, w12))
bigb= union[~union.isin(intersect)]
bigb


3      AACPS1
4      AACPS2
5      AACPS3
6      AACPS4
7      AACPS5
        ...  
120     TMDK1
121       TPI
122    UGLYCH
123     UM3PL
124      XPPT
Length: 94, dtype: object

In [3218]:
i=0
intersect_decr_below_zero_and_incr_above_zero= list()
intersect_decr_below_zero_and_decr_above_zero= list()
intersect_incr_below_zero_and_decr_above_zero= list()
intersect_incr_below_zero_and_incr_above_zero= list()
intersect_decr_below_zero_and_inANDdecr_above_zero= list()
intersect_incr_below_zero_and_inANDdecr_above_zero= list()
intersect_inANDdecr_below_zero_and_decr_above_zero= list()
intersect_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(intersect):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(intersect[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and intersect[i] not in intersect_decr_below_zero_and_decr_above_zero:
            intersect_decr_below_zero_and_decr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and intersect[i] not in intersect_incr_below_zero_and_incr_above_zero:
            intersect_incr_below_zero_and_incr_above_zero.append(intersect[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and intersect[i] not in intersect_decr_below_zero_and_incr_above_zero:
            intersect_decr_below_zero_and_incr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and intersect[i] not in intersect_incr_below_zero_and_decr_above_zero:
            intersect_incr_below_zero_and_decr_above_zero.append(intersect[i])
        elif my_series.is_monotonic_decreasing ==True and intersect[i] not in intersect_decr_below_zero_and_inANDdecr_above_zero:
            intersect_decr_below_zero_and_inANDdecr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and intersect[i] not in intersect_incr_below_zero_and_inANDdecr_above_zero:
            intersect_incr_below_zero_and_inANDdecr_above_zero.append(intersect[i])
        elif my_series2.is_monotonic_decreasing ==True and intersect[i] not in intersect_inANDdecr_below_zero_and_decr_above_zero:
            intersect_inANDdecr_below_zero_and_decr_above_zero.append(intersect[i])
        elif my_series2.is_monotonic ==True and intersect[i] not in intersect_inANDdecr_below_zero_and_incr_above_zero:
            intersect_inANDdecr_below_zero_and_incr_above_zero.append(intersect[i])
        
    i=i+1

intersect_decr_below_zero_and_decr_above_zero


[]

In [3219]:
intersect_incr_below_zero_and_decr_above_zero

['4PCP',
 '4PCPpp',
 '4PEPTabcpp',
 'AGM3PA',
 'AGM3PApp',
 'AGM3PH',
 'AGM3Pt2pp',
 'AGM4PCP',
 'AGM4PCPpp',
 'ALAALAabcpp',
 'AM4PA',
 'AM4PCP',
 'ARBTNR2',
 'ARBTNR3',
 'CPGNR2',
 'CPGNR3',
 'ETHAt2pp',
 'EX_ura_e',
 'EX_val__L_e',
 'FE3HOXR2',
 'FE3HOXR3',
 'FECRMR2',
 'FECRMR3',
 'FEENTERR2',
 'FEENTERR3',
 'FEOXAMR2',
 'FEOXAMR3',
 'G3PEabcpp',
 'GLYCtpp',
 'GPDDA4pp',
 'MLTGY2pp']

In [3220]:
i=0
bigb_decr_below_zero_and_incr_above_zero= list()
bigb_decr_below_zero_and_decr_above_zero= list()
bigb_incr_below_zero_and_decr_above_zero= list()
bigb_incr_below_zero_and_incr_above_zero= list()
bigb_decr_below_zero_and_inANDdecr_above_zero= list()
bigb_incr_below_zero_and_inANDdecr_above_zero= list()
bigb_inANDdecr_below_zero_and_decr_above_zero= list()
bigb_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(bigb):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(bigb.iloc[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_decr_above_zero:
            bigb_decr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_incr_above_zero:
            bigb_incr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_incr_above_zero:
            bigb_decr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_decr_above_zero:
            bigb_incr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_inANDdecr_above_zero:
            bigb_decr_below_zero_and_inANDdecr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_inANDdecr_above_zero:
            bigb_incr_below_zero_and_inANDdecr_above_zero.append(bigb.iloc[i])
        elif my_series2.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_inANDdecr_below_zero_and_decr_above_zero:
            bigb_inANDdecr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif my_series2.is_monotonic ==True and bigb.iloc[i] not in bigb_inANDdecr_below_zero_and_incr_above_zero:
            bigb_inANDdecr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        
    i=i+1

bigb_decr_below_zero_and_decr_above_zero


[]

In [3221]:
bigb_incr_below_zero_and_decr_above_zero

['AACPS1',
 'AACPS2',
 'AACPS3',
 'AACPS4',
 'AACPS5',
 'AACPS6',
 'AACPS7',
 'AGM4PA',
 'ARBTNR1',
 'ARGDC',
 'CLPNH120pp',
 'CLPNH140pp',
 'CLPNH141pp',
 'CLPNH160pp',
 'CLPNH161pp',
 'CLPNH180pp',
 'CLPNH181pp',
 'CPGNR1',
 'CYSDS',
 'CYSabcpp',
 'DALAt2pp',
 'DURIPP',
 'EAR100x',
 'EAR120x',
 'EAR121x',
 'EAR140x',
 'EAR141x',
 'EAR160x',
 'EAR161x',
 'EAR180x',
 'EAR181x',
 'ENTERES',
 'ENTERES2',
 'FA100ACPHi',
 'FA80ACPHi',
 'FACOAE100',
 'FACOAE141',
 'FE3HOXR1',
 'FECRMR1',
 'FEENTERR1',
 'FEOXAMR1',
 'FLVRx',
 'FMNRx',
 'G3PGabcpp',
 'GARFT',
 'GHMT2r',
 'GLCt2pp',
 'GMPR',
 'ICL',
 'IDOND',
 'LALDO2x',
 'LPLIPAL2E120',
 'LPLIPAL2E140',
 'LPLIPAL2E141',
 'LPLIPAL2E160',
 'LPLIPAL2E161',
 'LPLIPAL2E180',
 'LPLIPAL2E181',
 'LPLIPAL2G120',
 'LPLIPAL2G140',
 'LPLIPAL2G141',
 'LPLIPAL2G160',
 'LPLIPAL2G161',
 'LPLIPAL2G180',
 'LPLIPAL2G181',
 'NTD1',
 'P5CD',
 'PAPSR',
 'PAPSR2',
 'PGL',
 'PGPP120',
 'PGPP140',
 'PGPP141',
 'PGPP160',
 'PGPP161',
 'PGPP180',
 'PGPP181',
 'POR5',
 

In [3222]:
bigb_incr_below_zero_and_inANDdecr_above_zero

['FUM', 'GAPD', 'SUCDi']

In [3223]:
bigb_inANDdecr_below_zero_and_decr_above_zero

['EX_ala__D_e', 'EX_succ_e', 'PGK', 'PPS']

In [3224]:
bigb_inANDdecr_below_zero_and_decr_above_zero.remove("PPS")
bigb_inANDdecr_below_zero_and_decr_above_zero.remove("EX_ala__D_e")
bigb_inANDdecr_below_zero_and_decr_above_zero


['EX_succ_e', 'PGK']

In [3225]:
bigb_incr_below_zero_and_inANDdecr_above_zero.remove("FUM")
bigb_incr_below_zero_and_inANDdecr_above_zero.remove("SUCDi")
bigb_incr_below_zero_and_decr_above_zero.append("FUM")
bigb_incr_below_zero_and_decr_above_zero.append("SUCDi")
decreasing_below_zero_and_decreasing_above_zero.append("EX_ala__D_e")
bigb_incr_below_zero_and_decr_above_zero.append("PPS")


In [3226]:
i=0
inANDdecr_decr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(truly_inANDdecr_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(truly_inANDdecr_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_decr_above_zero:
            inANDdecr_decr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_incr_above_zero:
            inANDdecr_incr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_incr_above_zero:
            inANDdecr_decr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_decr_above_zero:
            inANDdecr_incr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_decr_below_zero_and_inANDdecr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_incr_below_zero_and_inANDdecr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_decr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_incr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        
    i=i+1

inANDdecr_decr_below_zero_and_decr_above_zero


[]

In [3227]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("EX_co2_e")
inANDdecr_incr_below_zero_and_decr_above_zero.append("EX_co2_e")
inANDdecr_incr_below_zero_and_decr_above_zero


['SSALy',
 'APH141',
 'SUCMALtpp',
 '2AGPGAT181',
 'NMNDA',
 'FACOAE180',
 'RNDR3',
 'NMNAT',
 'GUAD',
 '2AGPGAT160',
 'MALt2_2pp',
 'XYLI2',
 'NI2tpp',
 'DURIK1',
 '2AGPEAT161',
 'AKGDH',
 'MOX',
 '2AGPGAT140',
 'FTHFD',
 'MGSA',
 'NI2t3pp',
 '2AGPEAT181',
 'RNTR2c',
 'GDPDPK',
 '2AGPEAT120',
 'RNDR4b',
 'RNTR3c2',
 'GDPMNH',
 'CDAPPA141',
 '2AGPGAT120',
 'FBA',
 'CDAPPA140',
 'CYTK1',
 'APH180',
 'CYStpp',
 'APH140',
 'RNDR1b',
 'GDPTPDP',
 'NMNN',
 'RPE',
 'PA160abcpp',
 'ASNS2',
 'ORNDC',
 'PUNP3',
 'APH120',
 'ZN2tpp',
 'CBMKr',
 'GLUDC',
 'PA141abcpp',
 'RNTR4c',
 'PFL',
 '2AGPEAT160',
 'PGI',
 'NDPK8',
 'PUNP1',
 'NTD2',
 'RNTR4c2',
 'CYTDK2',
 'SUCCt2_2pp',
 'LDH_D',
 'MTHFC',
 'GLYCL',
 '2AGPGAT180',
 'RNTR1c',
 'GND',
 '2AGPEAT180',
 'GLYAT',
 'PPK2',
 'RNTR1c2',
 'PA161abcpp',
 'RNTR3c',
 'IMPD',
 'CDAPPA181',
 '2AGPEAT141',
 'PUNP2',
 'FACOAE160',
 'CDAPPA120',
 'NDPK5',
 'APH161',
 'FACOAE120',
 'RNDR4',
 'SUCASPtpp',
 'NTD11',
 'MTHFD',
 'NACODA',
 '2AGPGAT141',
 'TRE6PH'

In [3228]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['ENO', 'MDH', 'CYTBO3_4pp', 'ATPS4rpp']

In [3229]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("MDH")
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("ENO")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("GLUDy")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("FLDR2")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("THD2pp")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("PPCK")
inANDdecr_incr_below_zero_and_decr_above_zero.append("MDH")
inANDdecr_incr_below_zero_and_decr_above_zero.append("ENO")
inANDdecr_incr_below_zero_and_decr_above_zero.append("GLUDy")
inANDdecr_incr_below_zero_and_decr_above_zero.append("FLDR2")
inANDdecr_incr_below_zero_and_decr_above_zero.append("THD2pp")
inANDdecr_incr_below_zero_and_decr_above_zero.append("PPCK")


inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("FBP")
inANDdecr_incr_below_zero_and_decr_above_zero.append("FBP")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("GLYCDx")
inANDdecr_incr_below_zero_and_decr_above_zero.append("GLYCDx")


In [3230]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

['FE3abcpp',
 'H2Otpp',
 'CAT',
 'EX_pro__L_e',
 'CO2tpp',
 'EX_4abut_e',
 'CO2tex',
 'EX_glyc_e',
 'EX_ade_e',
 'EX_alltn_e',
 'EX_glyc__R_e',
 'EX_etoh_e',
 'PGM',
 'EX_idon__L_e',
 'EX_gua_e',
 'EX_12ppd__S_e',
 'EX_asn__L_e',
 'EX_xan_e']

In [3231]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['CYTBO3_4pp', 'ATPS4rpp']

In [3232]:
len(truly_inANDdecr_both_positive_and_negative)

171

In [3233]:
len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)

171

In [3234]:
increasing_below_zero_and_decreasing_above_zero= increasing_below_zero_and_decreasing_above_zero + intersect_incr_below_zero_and_decr_above_zero + bigb_incr_below_zero_and_decr_above_zero + inANDdecr_incr_below_zero_and_decr_above_zero
incr_below_zero_and_inANDdecr_above_zero= bigb_incr_below_zero_and_inANDdecr_above_zero + inANDdecr_incr_below_zero_and_inANDdecr_above_zero


In [3235]:
inANDdecr_below_zero_and_decr_above_zero= bigb_inANDdecr_below_zero_and_decr_above_zero + inANDdecr_inANDdecr_below_zero_and_decr_above_zero


In [3236]:
increasing_below_zero_and_decreasing_above_zero.append("F6PA")
increasing_below_zero_and_decreasing_above_zero.append("MACPD")
increasing_below_zero_and_decreasing_above_zero


['CITL',
 'PYK',
 '4PCP',
 '4PCPpp',
 '4PEPTabcpp',
 'AGM3PA',
 'AGM3PApp',
 'AGM3PH',
 'AGM3Pt2pp',
 'AGM4PCP',
 'AGM4PCPpp',
 'ALAALAabcpp',
 'AM4PA',
 'AM4PCP',
 'ARBTNR2',
 'ARBTNR3',
 'CPGNR2',
 'CPGNR3',
 'ETHAt2pp',
 'EX_ura_e',
 'EX_val__L_e',
 'FE3HOXR2',
 'FE3HOXR3',
 'FECRMR2',
 'FECRMR3',
 'FEENTERR2',
 'FEENTERR3',
 'FEOXAMR2',
 'FEOXAMR3',
 'G3PEabcpp',
 'GLYCtpp',
 'GPDDA4pp',
 'MLTGY2pp',
 'AACPS1',
 'AACPS2',
 'AACPS3',
 'AACPS4',
 'AACPS5',
 'AACPS6',
 'AACPS7',
 'AGM4PA',
 'ARBTNR1',
 'ARGDC',
 'CLPNH120pp',
 'CLPNH140pp',
 'CLPNH141pp',
 'CLPNH160pp',
 'CLPNH161pp',
 'CLPNH180pp',
 'CLPNH181pp',
 'CPGNR1',
 'CYSDS',
 'CYSabcpp',
 'DALAt2pp',
 'DURIPP',
 'EAR100x',
 'EAR120x',
 'EAR121x',
 'EAR140x',
 'EAR141x',
 'EAR160x',
 'EAR161x',
 'EAR180x',
 'EAR181x',
 'ENTERES',
 'ENTERES2',
 'FA100ACPHi',
 'FA80ACPHi',
 'FACOAE100',
 'FACOAE141',
 'FE3HOXR1',
 'FECRMR1',
 'FEENTERR1',
 'FEOXAMR1',
 'FLVRx',
 'FMNRx',
 'G3PGabcpp',
 'GARFT',
 'GHMT2r',
 'GLCt2pp',
 'GMPR',
 

In [3237]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
D4= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
D5= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (flat, columns = ['Reaction names'])
D8= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (flat_both_positive_and_negative, columns = ['Reaction names'])
D10= pd.DataFrame (increasing_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (flat_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (decreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D18= pd.DataFrame (increasing_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D19= pd.DataFrame (increasing_below_zero_and_increasing_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D21= pd.DataFrame (one_to_three_data_point, columns = ['Reaction names'])
D22= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D24= pd.DataFrame (truly_inANDdecr_below_zero, columns = ['Reaction names'])
D25= pd.DataFrame (decreasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (increasing_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])


In [3238]:
len(D)+len(D1)+len(D2)+len(D4)+len(D5)+len(D6)+len(D7)+len(D8)+len(D9)+len(D10)+len(D11)+len(D12)+len(D13)+len(D15)+len(D16)+len(D17)+len(D18)+len(D19)+len(D20)+len(D21)+len(D22)+len(D23)+len(D24)+len(D25)+len(D26)+len(D27)+len(D28)


2631

In [3239]:
i=0
one_to_three_data_point_below_zero= list()
one_to_three_data_point_above_zero= list()

while i < len(flat_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero.append(flat_both_positive_and_negative[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero.append(flat_both_positive_and_negative[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero


['ACACT4r', 'HACD4']

In [3240]:
one_to_three_data_point_above_zero

['CTECOAI6_copy1',
 'DHORD2',
 'DHORDfum',
 'EAR100y',
 'EAR121y',
 'EAR141y',
 'GLCtex_copy1',
 'MDDCP3pp',
 'MLDCP2App']

In [3241]:
i=0
F_below_zero_and_IorD_above_zero= list()
F_below_zero_and_ISorDS_above_zero= list()
IorD_below_zero_and_F_above_zero= list()
ISorDS_below_zero_and_F_above_zero= list()
F_below_zero_and_F_above_zero= list()
r_id ="ACACT8r_copy1"
csvFile = pd.read_csv('biomass_per_flux_for_reaction_ACACT8r_copy1.csv')
l= list()

while i < 101:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile2= pd.DataFrame (l, columns = ['Biomass'])
csvFile2.Biomass = csvFile2.Biomass.round(2)

my_series = csvFile2['Biomass'].squeeze()

if len(csvFile2['Biomass'])>3:
        
    if len(my_series.unique())>3 and len(my_series.unique())>len(my_series)-4 and r_id not in ISorDS_below_zero_and_F_above_zero:
        IorD_below_zero_and_F_above_zero.append(r_id)
    elif len(my_series.unique())>3 and len(my_series.unique())<len(my_series)-4 and r_id not in IorD_below_zero_and_F_above_zero:
        ISorDS_below_zero_and_F_above_zero.append(r_id)
    
l= list()
i=100
        
while i < 201:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile3= pd.DataFrame (l, columns = ['Biomass'])
csvFile3.Biomass = csvFile3.Biomass.round(2)

my_series2 = csvFile3['Biomass'].squeeze()

if len(csvFile3['Biomass'])>3:
        
    if len(my_series2.unique())>3 and len(my_series2.unique())>len(my_series2)-4 and r_id not in F_below_zero_and_ISorDS_above_zero:
        F_below_zero_and_IorD_above_zero.append(r_id)
    elif len(my_series2.unique())>3 and len(my_series2.unique())<len(my_series2)-4 and r_id not in F_below_zero_and_IorD_above_zero:
        F_below_zero_and_ISorDS_above_zero.append(r_id)
    else:
        if r_id not in (ISorDS_below_zero_and_F_above_zero or IorD_below_zero_and_F_above_zero):
            F_below_zero_and_F_above_zero.append(r_id)

F_below_zero_and_F_above_zero


[]

In [3242]:
F_below_zero_and_IorD_above_zero

['ACACT8r_copy1']

In [3243]:
set1= set(flat_both_positive_and_negative)
set2= set(one_to_three_data_point_below_zero)
set3= set(one_to_three_data_point_above_zero)
truly_flat_both_positive_and_negative= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_flat_above_zero= list(one_to_three_data_point_below_zero)
flat_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero)

i3= 0
F_below_zero_and_IorD_above_zero= list()
F_below_zero_and_ISorDS_above_zero= list()
IorD_below_zero_and_F_above_zero= list()
ISorDS_below_zero_and_F_above_zero= list()
F_below_zero_and_F_above_zero= list()
r_id = truly_flat_both_positive_and_negative[0]

for r_id in truly_flat_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = truly_flat_both_positive_and_negative[i3]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>3:
        
        if len(my_series.unique())>3 and len(my_series.unique())>len(my_series)-4 and r_id not in ISorDS_below_zero_and_F_above_zero:
            IorD_below_zero_and_F_above_zero.append(r_id)
        elif len(my_series.unique())>3 and len(my_series.unique())<len(my_series)-4 and r_id not in IorD_below_zero_and_F_above_zero:
            ISorDS_below_zero_and_F_above_zero.append(r_id)
    
    l= list()
    i=100
        
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile3= pd.DataFrame (l, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)

    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile3['Biomass'])>3:
        
        if len(my_series2.unique())>3 and len(my_series2.unique())>len(my_series2)-4 and r_id not in F_below_zero_and_ISorDS_above_zero:
            F_below_zero_and_IorD_above_zero.append(r_id)
        elif len(my_series2.unique())>3 and len(my_series2.unique())<len(my_series2)-4 and r_id not in F_below_zero_and_IorD_above_zero:
            F_below_zero_and_ISorDS_above_zero.append(r_id)
        else:
            if r_id not in (ISorDS_below_zero_and_F_above_zero or IorD_below_zero_and_F_above_zero):
                F_below_zero_and_F_above_zero.append(r_id)

    
    i3= i3 + 1

F_below_zero_and_F_above_zero = [*set(F_below_zero_and_F_above_zero)]
F_below_zero_and_F_above_zero


['GLDBRAN2',
 'GLYt4pp',
 'GLCP2',
 'ACACT8r_copy2',
 'ACt4pp',
 'HACD4i',
 'HACD8i',
 'ACACT13',
 'ADK1',
 'ICHORS_copy2',
 'HPYRRx',
 'ASPt2pp_copy1',
 'FEROpp',
 'INDOLEt2pp',
 'GLYCLTt4pp',
 'HACD1i',
 'KAT1',
 'MICITDr_copy2',
 'NAt3pp',
 'VPAMTr_copy1',
 'URAt2pp_copy1',
 'L_LACtex_copy2',
 'CRNt8pp',
 'HACD3i',
 'HACD5i',
 'ABUTt2pp',
 'ACACT8',
 'DARBt2rpp_copy2',
 'ACACT9',
 'ALATA_L',
 'CA2t3pp',
 'SERt4pp',
 'VALTA',
 'DHNPA2r_copy2',
 'ACACT10',
 'ACACT12',
 'GLCtex_copy2',
 'VPAMTr_copy2',
 'ALDD19xr_copy2',
 'ACS',
 'CTECOAI6_copy2',
 'GLCP',
 'CTECOAI8_copy2',
 'NDPK1',
 'ADK3',
 'ALAt2pp_copy1',
 'CTECOAI7_copy2',
 'GLYt2pp_copy1',
 'GLUt4pp',
 'L_LACtex_copy1',
 'ACACT11',
 'HACD2i',
 'PROt4pp',
 'ALAt4pp',
 'HACD7i',
 'HACD6i']

In [3244]:
I_below_zero_and_F_above_zero= list(IorD_below_zero_and_F_above_zero)
increasing_below_zero_and_flat_above_zero= list(increasing_below_zero_and_flat_above_zero) + list(I_below_zero_and_F_above_zero)

F_below_zero_and_D_above_zero= list(F_below_zero_and_IorD_above_zero)
flat_below_zero_and_decreasing_above_zero= list(flat_below_zero_and_decreasing_above_zero) + list(F_below_zero_and_D_above_zero)


In [3245]:
i=0
one_to_three_data_point_below_zero2= list()
one_to_three_data_point_above_zero2= list()

while i < len(increasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(increasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero2.append(increasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero2.append(increasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero2


[]

In [3246]:
one_to_three_data_point_above_zero2

['DHORD5']

In [3247]:
i=0
one_to_three_data_point_below_zero3= list()
one_to_three_data_point_above_zero3= list()

while i < len(decreasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(decreasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero3.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero3.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero3


[]

In [3248]:
one_to_three_data_point_above_zero3

['SULabcpp',
 'H2SO',
 'EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'GLCabcpp']

In [3249]:
i=0
one_to_three_data_point_below_zero4= list()
one_to_three_data_point_above_zero4= list()

while i < len(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero4.append(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero4.append(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero4


[]

In [3250]:
one_to_three_data_point_above_zero4

['EX_ac_e']

In [3251]:
i=0
one_to_three_data_point_below_zero5= list()
one_to_three_data_point_above_zero5= list()

while i < len(inANDdecr_below_zero_and_decr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_below_zero_and_decr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero5.append(inANDdecr_below_zero_and_decr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero5.append(inANDdecr_below_zero_and_decr_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero5


['PGK']

In [3252]:
one_to_three_data_point_above_zero5

['EX_succ_e',
 'EX_pro__L_e',
 'EX_4abut_e',
 'EX_glyc_e',
 'EX_ade_e',
 'EX_alltn_e',
 'EX_glyc__R_e',
 'EX_etoh_e',
 'EX_idon__L_e',
 'EX_gua_e',
 'EX_12ppd__S_e',
 'EX_asn__L_e',
 'EX_xan_e']

In [3253]:
i=0
one_to_three_data_point_below_zero6= list()
one_to_three_data_point_above_zero6= list()

while i < len(increasing_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(increasing_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero6.append(increasing_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero6.append(increasing_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero6


['4PCP',
 '4PCPpp',
 '4PEPTabcpp',
 'AGM3PA',
 'AGM3PApp',
 'AGM3PH',
 'AGM3Pt2pp',
 'AGM4PCP',
 'AGM4PCPpp',
 'ALAALAabcpp',
 'AM4PA',
 'AM4PCP',
 'ETHAt2pp',
 'MLTGY2pp',
 'AACPS2',
 'CYSDS',
 'CYSabcpp',
 'DALAt2pp',
 'DURIPP',
 'FA80ACPHi',
 'FACOAE141',
 'GMPR',
 'ICL',
 'NTD1',
 'P5CD',
 'PGL',
 'PROD2',
 'TKT1',
 'TKT2',
 'TMDK1',
 'XPPT',
 'FUM',
 'SUCDi']

In [3254]:
one_to_three_data_point_above_zero6

['4PCP',
 '4PCPpp',
 '4PEPTabcpp',
 'AGM3PA',
 'AGM3PApp',
 'AGM3PH',
 'AGM3Pt2pp',
 'AGM4PCP',
 'AGM4PCPpp',
 'ALAALAabcpp',
 'AM4PA',
 'AM4PCP',
 'ETHAt2pp',
 'EX_ura_e',
 'EX_val__L_e',
 'G3PEabcpp',
 'GLYCtpp',
 'MLTGY2pp',
 'AGM4PA',
 'EAR100x',
 'EAR121x',
 'EAR141x',
 'ENTERES2',
 'G3PGabcpp',
 'GARFT',
 'GLCt2pp',
 'IDOND',
 'LALDO2x',
 'PAPSR',
 'PAPSR2',
 'SO4t2pp',
 'TPI',
 'UGLYCH',
 'UM3PL']

In [3255]:
i=0
one_to_three_data_point_below_zero7= list()
one_to_three_data_point_above_zero7= list()

while i < len(decreasing_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(decreasing_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero7.append(decreasing_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero7.append(decreasing_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero7


[]

In [3256]:
one_to_three_data_point_above_zero7

['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e',
 'EX_ala__D_e']

In [3257]:
i=0
one_to_three_data_point_below_zero8= list()
one_to_three_data_point_above_zero8= list()

while i < len(increasing_below_zero_and_increasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(increasing_below_zero_and_increasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero8.append(increasing_below_zero_and_increasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero8.append(increasing_below_zero_and_increasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero8

[]

In [3258]:
one_to_three_data_point_above_zero8

['GLCptspp']

In [3259]:
i=0
one_to_three_data_point_below_zero9= list()
one_to_three_data_point_above_zero9= list()

while i < len(incr_below_zero_and_inANDdecr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(incr_below_zero_and_inANDdecr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero9.append(incr_below_zero_and_inANDdecr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero9.append(incr_below_zero_and_inANDdecr_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero9


[]

In [3260]:
one_to_three_data_point_above_zero9

['GAPD']

In [3261]:
i=0
one_to_three_data_point3_below_zero= list()
one_to_three_data_point3_above_zero= list()

while i < len(one_to_three_data_point3):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(one_to_three_data_point3[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point3_below_zero:
        
        one_to_three_data_point3_below_zero.append(one_to_three_data_point3[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point3_above_zero:
        
        one_to_three_data_point3_above_zero.append(one_to_three_data_point3[i])
    
    i= i + 1
    
one_to_three_data_point3_below_zero


['FACOAE80',
 'GNK',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp']

In [3262]:
one_to_three_data_point3_above_zero


['FACOAE80',
 'GNK',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp']

In [3263]:
i=0
one_to_three_data_point_below_zero10= list()
one_to_three_data_point_above_zero10= list()

while i < len(flat):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero10:
        
        one_to_three_data_point_below_zero10.append(flat[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero10:
        
        one_to_three_data_point_above_zero10.append(flat[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero10


[]

In [3264]:
one_to_three_data_point_above_zero10

['ACLS', 'DHAD1']

In [3265]:
i=0
one_to_three_data_point_below_zero11= list()
one_to_three_data_point_above_zero11= list()

while i < len(flat_below_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_below_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero11:
        
        one_to_three_data_point_below_zero11.append(flat_below_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero11:
        
        one_to_three_data_point_above_zero11.append(flat_below_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero11

['KARA1']

In [3266]:
one_to_three_data_point_above_zero11

[]

In [3267]:
set1= set(one_to_three_data_point3)
set2= set(one_to_three_data_point3_below_zero)
set3= set(one_to_three_data_point3_above_zero)
truly_one_to_three_data_point3= list(set1-set2-set3)
truly_one_to_three_data_point3


[]

In [3268]:
set1= set(increasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero2)
set3= set(one_to_three_data_point_above_zero2)
truly_increasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(decreasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero3)
set3= set(one_to_three_data_point_above_zero3)
truly_decreasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero4)
set3= set(one_to_three_data_point_above_zero4)
truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(inANDdecr_below_zero_and_decr_above_zero)
set2= set(one_to_three_data_point_below_zero5)
set3= set(one_to_three_data_point_above_zero5)
truly_inANDdecr_below_zero_and_decr_above_zero= list(set1-set2-set3)

set1= set(increasing_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero6)
set3= set(one_to_three_data_point_above_zero6)
truly_increasing_below_zero_and_decreasing_above_zero= list(set1-set2-set3)


set1= set(decreasing_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero7)
set3= set(one_to_three_data_point_above_zero7)
truly_decreasing_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(increasing_below_zero_and_increasing_above_zero)
set2= set(one_to_three_data_point_below_zero8)
set3= set(one_to_three_data_point_above_zero8)
truly_increasing_below_zero_and_increasing_above_zero= list(set1-set2-set3)

set1= set(incr_below_zero_and_inANDdecr_above_zero)
set2= set(one_to_three_data_point_below_zero9)
set3= set(one_to_three_data_point_above_zero9)
truly_incr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)

set1= set(one_to_three_data_point3)
set2= set(one_to_three_data_point3_below_zero)
set3= set(one_to_three_data_point3_above_zero)
truly_one_to_three_data_point3= list(set1-set2-set3)

set1= set(flat)
set2= set(one_to_three_data_point_below_zero10)
set3= set(one_to_three_data_point_above_zero10)
truly_flat_above_zero= list(set1-set2-set3)

set1= set(flat_below_zero)
set2= set(one_to_three_data_point_below_zero11)
set3= set(one_to_three_data_point_above_zero11)
truly_flat_below_zero= list(set1-set2-set3)

intersectI = pd.Series(np.intersect1d(one_to_three_data_point_below_zero, one_to_three_data_point_above_zero))
intersectII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero2, one_to_three_data_point_above_zero2))
intersectIII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero3, one_to_three_data_point_above_zero4))
intersectIV = pd.Series(np.intersect1d(one_to_three_data_point_below_zero4, one_to_three_data_point_above_zero4))
intersectV = pd.Series(np.intersect1d(one_to_three_data_point_below_zero5, one_to_three_data_point_above_zero5))
intersectVI = pd.Series(np.intersect1d(one_to_three_data_point_below_zero6, one_to_three_data_point_above_zero6))
intersectVII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero7, one_to_three_data_point_above_zero7))
intersectVIII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero8, one_to_three_data_point_above_zero8))
intersectIX = pd.Series(np.intersect1d(one_to_three_data_point_below_zero9, one_to_three_data_point_above_zero9))
intersectX = pd.Series(np.intersect1d(one_to_three_data_point3_below_zero, one_to_three_data_point3_above_zero))

truly_one_to_three_data_point_both_positive_and_negative= list(intersectVI)+list(intersectX)

set1= set(one_to_three_data_point_below_zero6)
set2= set(intersectVI)
one_to_three_data_point_below_zero6= list(set1-set2)

set3= set(one_to_three_data_point_above_zero6)
set4= set(intersectVI)
one_to_three_data_point_above_zero6= list(set3-set4)

truly_one_to_three_data_point_below_zero= list(one_to_three_data_point2)+list(one_to_three_data_point_below_zero11)
truly_one_to_three_data_point_above_zero= list(one_to_three_data_point)+list(one_to_three_data_point_above_zero10)


one_to_three_data_point_below_zero_and_decreasing_above_zero= list(one_to_three_data_point_below_zero2)+list(one_to_three_data_point_below_zero3)+list(one_to_three_data_point_below_zero4)+list(one_to_three_data_point_below_zero5)+list(one_to_three_data_point_below_zero6)+list(one_to_three_data_point_below_zero7)
increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero2)
decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero3)
increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero4)

inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero5)
increasing_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero6)+list(one_to_three_data_point_above_zero8)+list(one_to_three_data_point_above_zero9)
decreasing_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero7)

one_to_three_data_point_below_zero_and_increasing_above_zero= list(one_to_three_data_point_below_zero8)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(one_to_three_data_point_below_zero9)


In [3269]:
truly_inANDdecr_below_zero_and_decr_above_zero.remove("CO2tex")
truly_inANDdecr_below_zero_and_decr_above_zero.remove("PGM")
truly_inANDdecr_below_zero_and_decr_above_zero.remove("CO2tpp")

truly_increasing_below_zero_and_decreasing_above_zero.append("CO2tex")
truly_increasing_below_zero_and_decreasing_above_zero.append("PGM")
truly_increasing_below_zero_and_decreasing_above_zero.append("CO2tpp")



In [3270]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
D4= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
D5= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (truly_flat_above_zero, columns = ['Reaction names'])
D8= pd.DataFrame (truly_flat_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (F_below_zero_and_F_above_zero, columns = ['Reaction names'])
D10= pd.DataFrame (increasing_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (truly_increasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (flat_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (truly_decreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (truly_inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (truly_increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D18= pd.DataFrame (increasing_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D19= pd.DataFrame (increasing_below_zero_and_increasing_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D22= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D24= pd.DataFrame (truly_inANDdecr_below_zero, columns = ['Reaction names'])
D25= pd.DataFrame (truly_decreasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (truly_increasing_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (truly_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (truly_one_to_three_data_point_below_zero, columns = ['Reaction names'])
D30= pd.DataFrame (truly_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D31= pd.DataFrame (truly_one_to_three_data_point_both_positive_and_negative, columns = ['Reaction names'])
D32= pd.DataFrame (one_to_three_data_point_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D33= pd.DataFrame (flat_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D34= pd.DataFrame (one_to_three_data_point_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D35= pd.DataFrame (increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D36= pd.DataFrame (decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D37= pd.DataFrame (increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D38= pd.DataFrame (inANDdecr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D39= pd.DataFrame (increasing_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D40= pd.DataFrame (decreasing_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D41= pd.DataFrame (one_to_three_data_point_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D42= pd.DataFrame (one_to_three_data_point_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])


In [3271]:
len(D)+len(D1)+len(D2)+len(D4)+len(D5)+len(D6)+len(D7)+len(D8)+len(D9)+len(D10)+len(D11)+len(D12)+len(D13)+len(D15)+len(D16)+len(D17)+len(D18)+len(D19)+len(D20)+len(D22)+len(D23)+len(D24)+len(D25)+len(D26)+len(D27)+len(D28)+len(D29)+len(D30)+len(D31)+len(D32)+len(D33)+len(D34)+len(D35)+len(D36)+len(D37)+len(D38)+len(D39)+len(D40)+len(D41)+len(D42)


2726

In [3273]:
D.to_csv("NEW one feasible value at zero reactions.csv")
D1.to_csv("NEW increasing and stable above zero.csv")
D2.to_csv("NEW decreasing and stable above zero.csv")
D4.to_csv("NEW decreasing and stable below zero.csv")
D5.to_csv("NEW increasing and stable below zero.csv")
D6.to_csv("NEW increasing and decreasing and stable below zero.csv")
D7.to_csv("NEW flat above zero.csv")
D8.to_csv("NEW flat below zero.csv")
D9.to_csv("NEW flat both positive and negative.csv")
D10.to_csv("NEW decreasing below zero and flat above zero.csv")
D11.to_csv("NEW decreasing and stable below zero and decreasing above zero.csv")
D12.to_csv("NEW flat below zero and decreasing above zero.csv")
D13.to_csv("NEW increasing and stable only below zero and decreasing above zero.csv")
D15.to_csv("NEW increasing and decreasing and stable below zero and decreasing above zero.csv")
D16.to_csv("NEW increasing and decreasing below zero and decreasing above zero.csv")
D17.to_csv("NEW decreasing below zero and decreasing above zero.csv")
D18.to_csv("NEW decreasing below zero and decreasing and stable above zero.csv")
D19.to_csv("NEW decreasing below zero and increasing and decreasing and stable above zero.csv")
D20.to_csv("NEW decreasing above zero.csv")
D22.to_csv("NEW increasing below zero.csv")
D23.to_csv("NEW decreasing below zero.csv")
D24.to_csv("NEW increasing and decreasing below zero.csv")
D25.to_csv("NEW increasing below zero and decreasing above zero.csv")
D26.to_csv("NEW decreasing below zero and increasing above zero.csv")
D27.to_csv("NEW decreasing below zero and increasing and decreasing above zero.csv")
D28.to_csv("NEW decreasing and stable below zero and decreasing and stable above zero")
D29.to_csv("NEW truly_one_to_three_data_point_below_zero.csv")
D30.to_csv("NEW truly_one_to_three_data_point_above_zero.csv")
D31.to_csv("NEW truly_one_to_three_data_point_both_positive_and_negative.csv")
D32.to_csv("NEW one_to_three_data_point_below_zero_and_flat_above_zero.csv")
D33.to_csv("NEW flat_below_zero_and_one_to_three_data_point_above_zero.csv")
D34.to_csv("NEW one_to_three_data_point_below_zero_and_decreasing_above_zero.csv")
D35.to_csv("NEW decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero.csv")
D36.to_csv("NEW increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero.csv")
D37.to_csv("NEW increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero.csv")
D38.to_csv("NEW inANDdecr_below_zero_and_one_to_three_data_point_above_zero.csv")
D39.to_csv("NEW decreasing_below_zero_and_one_to_three_data_point_above_zero.csv")
D40.to_csv("NEW increasing_below_zero_and_one_to_three_data_point_above_zero.csv")
D41.to_csv("NEW one_to_three_data_point_below_zero_and_increasing_above_zero.csv")
D42.to_csv("NEW one_to_three_data_point_below_zero_and_inANDdecr_above_zero.csv")


In [3272]:
Dtotal= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_only_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_only_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_only_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_only_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_flat_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(F_below_zero_and_F_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_decreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_inANDdecr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_below_zero_and_decreasing_above_zero))   
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_increasing_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_inANDdecr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_decreasing_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_below_zero_and_increasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_incr_below_zero_and_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_increasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_inANDdecr_above_zero))

              
Dtotal.to_csv("NEWEST all reactions.csv")
